In [ ]:
!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import backend
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score
from rouge import Rouge
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
# from pymeteor import meteor_score
from nltk.translate.meteor_score import meteor_score
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Embedding, TimeDistributed, Dense, Concatenate
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
# from bleu import Bleu
# from meteor import Meteor

In [ ]:
# Define the hyperparameters
BATCH_SIZE = 128
EMBEDDING_DIM = 300
EPOCHS = 1
MAX_LENGTH = 50
SUMMARY_MAX_LENGTH = 10
VOCAB_SIZE = 10000
TEXT_MAX_LENGTH = 500

In [ ]:
def load_data(file_path):
    df = pd.read_csv(file_path, encoding = 'ISO-8859-1')
    # df.SentimentText=df.SentimentText.astype(str)
    # df.characters=df.characters.astype(str)
    return df['text'].fillna({'data':''}).values, df['ctext'].fillna({'data':''}).values

In [ ]:
def tokenize(texts, summaries):
    texts = [str(text) for text in texts]
    summaries = [str(summary) for summary in summaries]
    text_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
    text_tokenizer.fit_on_texts(texts)
    text_sequences = text_tokenizer.texts_to_sequences(texts)
    
    summary_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=VOCAB_SIZE, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
    summary_tokenizer.fit_on_texts(summaries)
    summary_sequences = summary_tokenizer.texts_to_sequences(summaries)
    
    return text_tokenizer, summary_tokenizer
    #text_sequences, summary_sequences


In [ ]:
# Pad the sequences
# def pad_sequences(text_sequences, summary_sequences, max_text_length, max_summary_length):
#     padded_text_sequences = tf.keras.preprocessing.sequence.pad_sequences(text_sequences, maxlen=max_text_length, padding='post', truncating='post')
#     padded_summary_sequences = tf.keras.preprocessing.sequence.pad_sequences(summary_sequences, maxlen=max_summary_length, padding='post', truncating='post')
#     return padded_text_sequences, padded_summary_sequences

In [ ]:
def pad_sequences(text_sequences, summary_sequences, max_text_length, max_summary_length, tokenizer):
    text_sequences = tokenizer.texts_to_sequences(text_sequences)
    summary_sequences = tokenizer.texts_to_sequences(summary_sequences)
    padded_text_sequences = tf.keras.preprocessing.sequence.pad_sequences(text_sequences, maxlen=max_text_length, padding='post', truncating='post')
    padded_summary_sequences = tf.keras.preprocessing.sequence.pad_sequences(summary_sequences, maxlen=max_summary_length, padding='post', truncating='post')
    return padded_text_sequences, padded_summary_sequences

In [ ]:
# def pad_sequences(sequences, max_length, padding='post', truncating='post', value=0):
#     padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
#         sequences,
#         maxlen=max_length,
#         padding=padding,
#         truncating=truncating,
#         value=value
#     )
#     return padded_sequences

In [ ]:
# Compute the rouge score
def compute_rouge(ref, hyp):
    rouge = Rouge()
    scores = rouge.get_scores(hyp, ref)
    return scores[0]['rouge-1']['f']

In [ ]:
# Compute the bleu score
def compute_bleu(ref, hyp):
    ref = [ref.split()]
    hyp = hyp.split()
    return sentence_bleu(ref, hyp)

In [ ]:
def compute_meteor(ref, hyp):
    return single_meteor_score(ref, hyp)

In [ ]:
def train_validation_split(text_sequences, summary_sequences, validation_split=0.2):
  split_index = int(len(text_sequences) * (1 - validation_split))
  train_text_sequences = text_sequences[:split_index]
  train_summary_sequences = summary_sequences[:split_index]
  validation_text_sequences = text_sequences[split_index:]
  validation_summary_sequences = summary_sequences[split_index:]
  return train_text_sequences, train_summary_sequences, validation_text_sequences, validation_summary_sequences


In [ ]:
# def build_model(num_encoder_tokens, num_decoder_tokens, embedding_dim, max_text_length, max_summary_length):
#     encoder_inputs = Input(shape=(max_text_length,))
#     encoder_embedding = Embedding(num_encoder_tokens, embedding_dim, mask_zero=True)(encoder_inputs)
#     encoder_lstm = Bidirectional(LSTM(units=128, return_sequences=True, return_state=True, recurrent_dropout=0.2))
#     encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_embedding)
#     state_h = Concatenate()([forward_h, backward_h])
#     state_c = Concatenate()([forward_c, backward_c])
#     encoder_states = [state_h, state_c]

#     decoder_inputs = Input(shape=(max_summary_length,))
#     decoder_embedding = Embedding(num_decoder_tokens, embedding_dim, mask_zero=True)(decoder_inputs)
#     decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True, recurrent_dropout=0.2)
#     decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
#     decoder_dense = Dense(num_decoder_tokens, activation='softmax')
#     decoder_outputs = decoder_dense(decoder_outputs)

#     model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#     model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
#     return model

In [ ]:
def build_model(vocab_size):
    # Encoder input
    encoder_input = Input(shape=(None,))

    # Encoder embedding layer
    encoder_embedding_layer = Embedding(vocab_size, 128, mask_zero=True)
    encoder_embedding = encoder_embedding_layer(encoder_input)

    # Encoder LSTM layer
    encoder_lstm = Bidirectional(LSTM(units=64, return_sequences=True, return_state=True, recurrent_dropout=0.2))
    encoder_output, state_h, state_c, state_h2, state_c2 = encoder_lstm(encoder_embedding)

    # Concatenate the states of the bidirectional LSTM
    state_h = Concatenate()([state_h, state_h2])
    state_c = Concatenate()([state_c, state_c2])

    # Decoder input
    decoder_input = Input(shape=(None,))

    # Decoder embedding layer
    decoder_embedding_layer = Embedding(vocab_size, 128, mask_zero=True)
    decoder_embedding = decoder_embedding_layer(decoder_input)

    # Decoder LSTM layer
    decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
    decoder_output, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=[state_h, state_c])

    # Decoder output layer
    decoder_dense = Dense(vocab_size, activation='softmax')
    decoder_output = decoder_dense(decoder_output)

    # Define the model
    model = Model([encoder_input, decoder_input], decoder_output)
    model.compile(optimizer='adam', loss='categorical_crossentropy')

    return model

In [ ]:
def train_model(model, train_text_sequences, train_summary_sequences, validation_text_sequences, validation_summary_sequences, batch_size, epochs):
    train_summary_sequences = train_summary_sequences.reshape((*train_summary_sequences.shape, 1))
    validation_summary_sequences = validation_summary_sequences.reshape((*validation_summary_sequences.shape, 1))
    
    for i in range(epochs):
        model.fit([train_text_sequences, train_summary_sequences], train_summary_sequences, batch_size=batch_size, epochs=1, validation_data=([validation_text_sequences, validation_summary_sequences], validation_summary_sequences))
        
        # Print the model summary after each epoch
        print("Epoch {}/{}".format(i + 1, epochs))
        print("-" * 20)
        model.summary()
        print("\n")
        
    return model


In [ ]:
# def train_model(model, train_text_sequences, train_summary_sequences, validation_text_sequences, validation_summary_sequences):
#     # Define the optimizer and loss function
#     optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
#     loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

#     # Compile the model
#     model.compile(optimizer=optimizer, loss=loss_fn)

#     # Define the early stopping criteria
#     early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

#     # Train the model
#     history = model.fit(x=train_text_sequences, y=tf.one_hot(train_summary_sequences, depth=VOCAB_SIZE),
#                         validation_data=(validation_text_sequences, tf.one_hot(validation_summary_sequences, depth=VOCAB_SIZE)),
#                         batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping])

#     return history

In [ ]:
# def train_model(model, train_text_sequences, train_summary_sequences, validation_text_sequences, validation_summary_sequences):
#     # Train the model
#     model.fit(
#         x=[train_text_sequences, train_summary_sequences[:, :-1]],
#         y=train_summary_sequences[:, 1:],
#         validation_data=([validation_text_sequences, validation_summary_sequences[:, :-1]], validation_summary_sequences[:, 1:]),
#         epochs=EPOCHS,
#         batch_size=BATCH_SIZE,
#         callbacks=[EarlyStopping(monitor='val_loss', patience=2)]
#     )


In [ ]:
# def evaluate_model(model, text_tokenizer, text_sequences, summary_tokenizer, summary_sequences, batch_size):
#   rouge_scores = []
#   bleu_scores = []
#   meteor_scores = []
#   for i in range(len(text_sequences) // batch_size + 1):
#     text_batch = text_sequences[i * batch_size : (i + 1) * batch_size]
#     summary_batch = summary_sequences[i * batch_size : (i + 1) * batch_size]
#     prediction = model.predict([text_batch, summary_batch])
    
#     for j in range(len(text_batch)):
#         predicted_summary = ''
#         for word_index in prediction[j]:
#             word = summary_tokenizer.index_word.get(word_index, '<unk>')
#             if word == '<end>':
#                 break
#             predicted_summary += ' ' + word
        
#         reference = ''
#         for word_index in summary_batch[j]:
#             word = summary_tokenizer.index_word.get(word_index, '<unk>')
#             if word == '<end>':
#                 break
#             reference += ' ' + word
        
#         rouge_scores.append(compute_rouge(reference, predicted_summary))
#         bleu_scores.append(compute_bleu(reference, predicted_summary))
#         meteor_scores.append(compute_meteor(reference, predicted_summary))
        
#   avg_rouge_score = np.mean(rouge_scores)
#   avg_bleu_score = np.mean(bleu_scores)
#   avg_meteor_score = np.mean(meteor_scores)

#   return avg_rouge_score, avg_bleu_score, avg_meteor_score

In [ ]:
def split_data(padded_text_sequences, padded_summary_sequences):
    num_samples = len(padded_text_sequences)
    num_train_samples = int(0.8 * num_samples)
    train_text_sequences = padded_text_sequences[:num_train_samples]
    train_summary_sequences = padded_summary_sequences[:num_train_samples]
    validation_text_sequences = padded_text_sequences[num_train_samples:]
    validation_summary_sequences = padded_summary_sequences[num_train_samples:]
    return train_text_sequences, train_summary_sequences, validation_text_sequences, validation_summary_sequences

In [ ]:
def decode_sequence(model, text_tokenizer, text_sequence):
    reverse_target_word_index = text_tokenizer.index_word
    reverse_source_word_index = text_tokenizer.index_word
    target_word_index = text_tokenizer.word_index

    # Encode the input sequence
    encoder_input = np.array(text_sequence).reshape(1, len(text_sequence))

    # Get the states of the encoder
    _, h, c = model.get_layer('encoder')(encoder_input)

    # Generate an empty target sequence
    target_sequence = np.zeros((1, 1))

    # Populate the first character of target sequence with the start character
    target_sequence[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = model.get_layer('decoder')([target_sequence, h, c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_target_word_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_word

        # Exit condition: either hit max length or find stop character
        if (sampled_word == 'eostok' or len(decoded_sentence) > SUMMARY_MAX_LENGTH):
            stop_condition = True

        # Update the target sequence
        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = sampled_token_index

    return decoded_sentence


In [ ]:
# def decode_sequence(model, text_tokenizer, text_sequence):
#     reverse_target_word_index = text_tokenizer.index_word
#     reverse_source_word_index = text_tokenizer.index_word
#     target_word_index = text_tokenizer.word_index

#     # Encode the input sequence
#     encoder_input = np.array(text_sequence).reshape(1, len(text_sequence))

#     # Get the states of the encoder
#     states_value = model.encoder.predict(encoder_input)

#     # Generate an empty target sequence
#     target_sequence = np.zeros((1, 1))

#     # Populate the first character of target sequence with the start character
#     target_sequence[0, 0] = target_word_index['sostok']

#     stop_condition = False
#     decoded_sentence = ''
#     while not stop_condition:
#         output_tokens, h, c = model.decoder.predict([target_sequence] + states_value)

#         # Sample a token
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         sampled_word = reverse_target_word_index[sampled_token_index]
#         decoded_sentence += ' ' + sampled_word

#         # Exit condition: either hit max length or find stop character
#         if (sampled_word == 'eostok' or
#            len(decoded_sentence) > SUMMARY_MAX_LENGTH):
#             stop_condition = True

#         # Update the target sequence
#         target_sequence = np.zeros((1, 1))
#         target_sequence[0, 0] = sampled_token_index

#         # Update states
#         states_value = [h, c]

#     return decoded_sentence


In [ ]:
def decode_summary_sequence(summary_tokenizer, summary_sequence):
    index_to_word = {index: word for word, index in summary_tokenizer.word_index.items()}
    decoded_sequence = ''
    for index in summary_sequence:
        if index == 0:
            break
        decoded_sequence += ' ' + index_to_word[index]
    return decoded_sequence

In [ ]:
# def evaluate_model(model, text_tokenizer, text_sequences, summary_tokenizer, summary_sequences, batch_size):
#     rouge = Rouge()
#     # bleu = Bleu()
#     # meteor = meteor_score.meteor_score(predicted_summaries, actual_summaries)
    
#     avg_rouge_score = 0
#     avg_bleu_score = 0
#     avg_meteor_score = 0
#     for i in range(0, len(text_sequences), batch_size):
#         text_sequence = text_sequences[i:i+batch_size]
#         summary_sequence = summary_sequences[i:i+batch_size]
#         predicted_summaries = decode_sequence(model, text_tokenizer, text_sequence)
#         actual_summaries = decode_summary_sequence(summary_tokenizer, summary_sequence)
#         rouge_scores = rouge.get_scores(predicted_summaries, actual_summaries)
#         bleu_scores = sentence_bleu(predicted_summaries, actual_summaries)
#         meteor_scores = meteor_score.meteor_scores(predicted_summaries, actual_summaries)
#         avg_rouge_score += sum([rouge_score['rouge-1']['f'] for rouge_score in rouge_scores]) / len(rouge_scores)
#         avg_bleu_score += sum([bleu_score['bleu-4']['f'] for bleu_score in bleu_scores]) / len(bleu_scores)
#         avg_meteor_score += sum([meteor_score['meteor'] for meteor_score in meteor_scores]) / len(meteor_scores)
    
#     avg_rouge_score /= len(text_sequences) / batch_size
#     avg_bleu_score /= len(text_sequences) / batch_size
#     avg_meteor_score /= len(text_sequences) / batch_size
    
#     return avg_rouge_score, avg_bleu_score, avg_meteor_score

In [ ]:
# def evaluate_model(model, text_tokenizer, text_sequences, summary_tokenizer, summary_sequences, batch_size):
#     rouge = Rouge()
#     # bleu = Bleu()
#     # meteor = meteor_score.meteor_score(predicted_summaries, actual_summaries)
    
#     avg_rouge_score = 0
#     avg_bleu_score = 0
#     avg_meteor_score = 0
#     for i in range(0, len(text_sequences), batch_size):
#         text_sequence = text_sequences[i:i+batch_size]
#         summary_sequence = summary_sequences[i:i+batch_size]
        
#         # Extract the encoder model from the main model
#         encoder = Model(inputs=model.input[0], outputs=model.get_layer('encoder').output)
        
#         # Encode the input text sequence
#         encoded_text = encoder.predict(text_sequence)
        
#         # Decode the summary sequence
#         predicted_summaries = decode_sequence(model, text_tokenizer, text_sequence)
#         actual_summaries = decode_summary_sequence(summary_tokenizer, summary_sequence)
        
#         rouge_scores = rouge.get_scores(predicted_summaries, actual_summaries)
#         bleu_scores = sentence_bleu(predicted_summaries, actual_summaries)
#         meteor_scores = meteor_score.meteor_scores(predicted_summaries, actual_summaries)
#         avg_rouge_score += sum([rouge_score['rouge-1']['f'] for rouge_score in rouge_scores]) / len(rouge_scores)
#         avg_bleu_score += sum([bleu_score['bleu-4']['f'] for bleu_score in bleu_scores]) / len(bleu_scores)
#         avg_meteor_score += sum([meteor_score['meteor'] for meteor_score in meteor_scores]) / len(meteor_scores)
    
#     avg_rouge_score /= len(text_sequences) / batch_size
#     avg_bleu_score /= len(text_sequences) / batch_size
#     avg_meteor_score /= len(text_sequences) / batch_size
    
#     return avg_rouge_score, avg_bleu_score, avg_meteor_score

In [ ]:
# def evaluate_model(model, text_tokenizer, text_sequences, summary_tokenizer, summary_sequences, batch_size):
#     rouge = Rouge()
#     avg_rouge_score = 0
#     for i in range(0, len(text_sequences), batch_size):
#         text_sequence = text_sequences[i:i+batch_size]
#         summary_sequence = summary_sequences[i:i+batch_size]
        
#         # Access the encoder layer by name
#         encoder = model.get_layer('bidirectional_3')
        
#         # Get the encoder output
#         encoder_output, encoder_hidden_state, encoder_cell_state = encoder.predict(text_sequence)
        
#         # Get the decoder input
#         decoder_input = np.zeros((len(summary_sequence), 1))
#         decoder_input[:, 0] = summary_tokenizer.word_index['start']
        
#         # Initialize the decoder hidden state and cell state with the encoder hidden state and cell state
#         decoder_hidden_state = encoder_hidden_state
#         decoder_cell_state = encoder_cell_state
        
#         # Initialize the predicted summary with the start token
#         predicted_summary = ['start']
        
#         # Decode each token in the summary sequence
#         for j in range(summary_sequence.shape[1]):
#             # Predict the next token
#             decoder_output, decoder_hidden_state, decoder_cell_state = model.get_layer('decoder_lstm')(decoder_input, initial_state=[decoder_hidden_state, decoder_cell_state])
#             decoder_output = model.get_layer('decoder_dense')(decoder_output)
#             decoder_input = np.argmax(decoder_output, axis=2)
            
#             # Add the predicted token to the summary
#             predicted_summary.append(summary_tokenizer.index_word[decoder_input[0][0]])
            
#             # Stop decoding if the end token is predicted
#             if predicted_summary[-1] == 'end':
#                 break
        
#         # Convert the predicted summary to a string
#         predicted_summary = ' '.join(predicted_summary[1:-1])
        
#         # Get the actual summary
#         actual_summary = ' '.join([summary_tokenizer.index_word[token] for token in summary_sequence[i]])
        
#         # Compute the Rouge score
#         scores = rouge.get_scores(predicted_summary, actual_summary)
#         avg_rouge_score += scores[0]['rouge-1']['f']
    
#     avg_rouge_score /= len(text_sequences) / batch_size
    
#     return avg_rouge_score


In [ ]:
# def evaluate_model(model, text_tokenizer, text_sequences, summary_tokenizer, summary_sequences, batch_size):
#     rouge = Rouge()
#     avg_rouge_score = 0
#     encoder_layer_index = 3
#     decoder_lstm_layer_index = 7
#     decoder_dense_layer_index = 8
    
#     for i in range(0, len(text_sequences), batch_size):
#         text_sequence = text_sequences[i:i+batch_size]
#         summary_sequence = summary_sequences[i:i+batch_size]
        
#         # Access the encoder layer by index
#         encoder_output, encoder_hidden_state, encoder_cell_state = model.layers[encoder_layer_index](keras.backend.constant(text_sequence))
        
#         # Get the decoder input
#         decoder_input = np.zeros((len(summary_sequence), 1))
#         decoder_input[:, 0] = summary_tokenizer.word_index['start']
        
#         # Initialize the decoder hidden state and cell state with the encoder hidden state and cell state
#         decoder_hidden_state = encoder_hidden_state
#         decoder_cell_state = encoder_cell_state
        
#         # Initialize the predicted summary with the start token
#         predicted_summary = ['start']
        
#         # Decode each token in the summary sequence
#         for j in range(summary_sequence.shape[1]):
#             # Predict the next token
#             decoder_output, decoder_hidden_state, decoder_cell_state = model.layers[decoder_lstm_layer_index](decoder_input, initial_state=[decoder_hidden_state, decoder_cell_state])
#             decoder_output = model.layers[decoder_dense_layer_index](decoder_output)
#             decoder_input = np.argmax(decoder_output, axis=2)
            
#             # Add the predicted token to the summary
#             predicted_summary.append(summary_tokenizer.index_word[decoder_input[0][0]])
            
#             # Stop decoding if the end token is predicted
#             if predicted_summary[-1] == 'end':
#                 break
        
#         # Convert the predicted summary to a string
#         predicted_summary = ' '.join(predicted_summary[1:-1])
        
#         # Get the actual summary
#         actual_summary = ' '.join([summary_tokenizer.index_word[token] for token in summary_sequence[i]])
        
#         # Compute the Rouge score
#         scores = rouge.get_scores(predicted_summary, actual_summary)
#         avg_rouge_score += scores[0]['rouge-1']['f']
    
#     avg_rouge_score /= len(text_sequences) / batch_size
    
#     return avg_rouge_score


In [ ]:
# import tensorflow as tf

# def evaluate_model(model, text_tokenizer, text_sequences, summary_tokenizer, summary_sequences, batch_size):
#     rouge = Rouge()
#     avg_rouge_score = 0
#     for i in range(0, len(text_sequences), batch_size):
#         text_sequence = text_sequences[i:i+batch_size]
#         summary_sequence = summary_sequences[i:i+batch_size]

#         # Get the encoder layers by type
#         encoder_layers = [layer for layer in model.layers if isinstance(layer, keras.layers.Bidirectional) or isinstance(layer, keras.layers.LSTM)]

#         # Get the forward and backward layers
#         forward_layer = None
#         backward_layer = None
#         for layer in encoder_layers:
#             if isinstance(layer, keras.layers.Bidirectional):
#                 forward_layer = layer.forward_layer
#                 backward_layer = layer.backward_layer
#             else:
#                 forward_layer = layer
#                 backward_layer = None

#         # Get the encoder output
#         forward_output, forward_state_h, forward_state_c = tf.keras.backend.function(inputs=[model.input], outputs=[forward_layer.output, forward_layer.states[0], forward_layer.states[1]])(text_sequence)
#         if backward_layer is not None:
#             backward_output, backward_state_h, backward_state_c = tf.keras.backend.function(inputs=[model.input], outputs=[backward_layer.output, backward_layer.states[0], backward_layer.states[1]])(text_sequence[:, ::-1])
#             state_h = Concatenate()([forward_state_h, backward_state_h])
#             state_c = Concatenate()([forward_state_c, backward_state_c])
#             encoder_output = Concatenate()([forward_output, backward_output[:, ::-1]])
#         else:
#             state_h = forward_state_h
#             state_c = forward_state_c
#             encoder_output = forward_output

#         # Get the decoder input
#         decoder_input = np.zeros((len(summary_sequence), 1))
#         decoder_input[:, 0] = summary_tokenizer.word_index['start']

#         # Initialize the decoder hidden state and cell state with the encoder hidden state and cell state
#         decoder_hidden_state = state_h
#         decoder_cell_state = state_c

#         # Initialize the predicted summary with the start token
#         predicted_summary = ['start']

#         # Get the decoder LSTM layer by type
#         decoder_lstm = [layer for layer in model.layers if isinstance(layer, keras.layers.LSTM)][0]

#         # Get the decoder dense layer by type
#         decoder_dense = [layer for layer in model.layers if isinstance(layer, keras.layers.Dense)][0]

#         # Decode each token in the summary sequence
#         for j in range(summary_sequence.shape[1]):
#             # Predict the next token
#             decoder_output, decoder_hidden_state, decoder_cell_state = decoder_lstm(decoder_input, initial_state=[decoder_hidden_state, decoder_cell_state])
#             decoder_output = decoder_dense(decoder_output)
#             decoder_input = np.argmax(decoder_output, axis=2)

#             # Add the predicted token to the summary
#             predicted_summary.append(summary_tokenizer.index_word[decoder_input[0][0]])

#             # Stop decoding if the end token is predicted
#             if predicted_summary[-1] == 'end':
#                 break

#         # Convert the predicted summary to a string
#         predicted_summary = ' '.join(predicted_summary[1:-1])

#         # Get the actual summary
#         actual_summary = ' '.join([summary_tokenizer.index_word[token] for token in summary_sequence[i]])

#         # Compute the Rouge score
#         scores = rouge.get_scores(predicted_summary, actual_summary)
#         avg_rouge_score += scores[0]['rouge-1']['f']

#     avg_rouge_score /= len(text_sequences) / batch_size

#     return avg_rouge_score


In [ ]:
def evaluate_model(model, text_tokenizer, text_sequences, summary_tokenizer, summary_sequences, batch_size):
    rouge = Rouge()
    total_rouge_1 = 0
    total_rouge_2 = 0
    total_rouge_l = 0
    predicted_summaries = []
    actual_summaries = []
    
    for i in range(0, len(text_sequences), batch_size):
        text_sequence = np.array(text_sequences[i:i+batch_size])
        text_sequence = text_sequence[:, np.newaxis]
        text_sequence = tf.constant(text_sequence)
        summary_sequence = np.array(summary_sequences[i:i+batch_size])
        # summary_sequence = np.array(summary_sequence[:, np.newaxis])
        
        # Get the encoder layers by type
        encoder_layers = [layer for layer in model.layers if isinstance(layer, keras.layers.Bidirectional) or isinstance(layer, keras.layers.LSTM)]

        # Get the forward and backward layers
        forward_layer = None
        backward_layer = None
        for layer in encoder_layers:
            if isinstance(layer, keras.layers.Bidirectional):
                forward_layer = layer.forward_layer
                backward_layer = layer.backward_layer
            else:
                forward_layer = layer
                backward_layer = None

        # Get the encoder output
        # forward_output, forward_state_h, forward_state_c = forward_layer.predict(text_sequence)
        input_length = text_sequence.shape[1]
        text_sequence_list = [tf.convert_to_tensor(seq) for seq in text_sequence]
        forward_output, forward_state_h, forward_state_c = backend.rnn(forward_layer.cell,
                                                         text_sequence_list,
                                                         initial_states=forward_layer.get_initial_state(text_sequence_list),
                                                         input_length=input_length)
        if backward_layer is not None:
            # backward_output, backward_state_h, backward_state_c = backward_layer.predict(text_sequence[:, ::-1])
            backward_output, backward_state_h, backward_state_c = backend.rnn(backward_layer.cell,
                                                                 text_sequence[:, ::-1],
                                                                 initial_states=backward_layer.get_initial_state(text_sequence[:, ::-1]),
                                                                 input_length=input_length)
            state_h = Concatenate()([forward_state_h, backward_state_h])
            state_c = Concatenate()([forward_state_c, backward_state_c])
            encoder_output = Concatenate()([forward_output, backward_output[:, ::-1]])
        else:
            state_h = forward_state_h
            state_c = forward_state_c
            encoder_output = forward_output

        # Get the decoder input
        decoder_input = np.zeros((len(summary_sequence), 1))
        decoder_input[:, 0] = summary_tokenizer.word_index['start']

        # Initialize the decoder hidden state and cell state with the encoder hidden state and cell state
        decoder_hidden_state = state_h
        decoder_cell_state = state_c

        # Initialize the predicted summary with the start token
        predicted_summary = ['start']

        # Get the decoder LSTM layer by type
        decoder_lstm = [layer for layer in model.layers if isinstance(layer, keras.layers.LSTM)][0]

        # Get the decoder dense layer by type
        decoder_dense = [layer for layer in model.layers if isinstance(layer, keras.layers.Dense)][0]

        for j in range(summary_sequence.shape[1]):
          # Get the one-hot encoded form of the current decoder input
          onehot_decoder_input = to_categorical(decoder_input, num_classes=len(summary_tokenizer.word_index))

          # Use the decoder LSTM layer to get the decoder output and hidden states
          decoder_output, decoder_hidden_state, decoder_cell_state = decoder_lstm(onehot_decoder_input, initial_state=[decoder_hidden_state, decoder_cell_state])

          # Use the decoder dense layer to get the predicted next token
          decoder_token_output = decoder_dense(decoder_output)
          predicted_token_index = np.argmax(decoder_token_output, axis=2)
          predicted_token = summary_tokenizer.index_word[predicted_token_index[0][0]]

          # Add the predicted token to the predicted summary
          predicted_summary.append(predicted_token)

          # If the predicted token is the end token, stop decoding
          if predicted_token == 'end':
              break

          # Otherwise, set the current decoder input to the predicted token index
          decoder_input[0][0] = predicted_token_index[0][0]

        # Convert the predicted and actual summaries from tokenized form to string form
        predicted_summary = ' '.join(predicted_summary[1:-1])
        actual_summary = ' '.join([summary_tokenizer.index_word[token] for token in summary_sequence[0] if token > 0][1:-1])

        # Add the predicted and actual summaries to the list of summaries
        predicted_summaries.append(predicted_summary)
        actual_summaries.append(actual_summary)

        # Calculate the ROUGE scores for the predicted and actual summaries
        scores = rouge.get_scores(predicted_summary, actual_summary)[0]
        total_rouge_1 += scores['rouge-1']['f']
        total_rouge_2 += scores['rouge-2']['f']
        total_rouge_l += scores['rouge-l']['f']

    # Calculate the average ROUGE scores for all summaries
    avg_rouge_1 = total_rouge_1 / len(predicted_summaries)
    avg_rouge_2 = total_rouge_2 / len(predicted_summaries)
    avg_rouge_l = total_rouge_l / len(predicted_summaries)

    return avg_rouge_1, avg_rouge_2, avg_rouge_l, predicted_summaries, actual_summaries


In [ ]:
# def evaluate_model(model, text_tokenizer, text_sequences, summary_tokenizer, summary_sequences, batch_size):
#     # create batches of input data for evaluation
#     steps = len(text_sequences) // batch_size
#     if len(text_sequences) % batch_size != 0:
#         steps += 1

#     rouge_scores = []
#     bleu_scores = []
#     meteor_scores = []

#     for i in range(steps):
#         start = i * batch_size
#         end = (i + 1) * batch_size

#         # pad the batch to the maximum length
#         padded_text_sequences = tf.keras.preprocessing.sequence.pad_sequences(
#             text_sequences[start:end], maxlen=MAX_LENGTH, padding='post', truncating='post'
#         )
#         padded_summary_sequences = tf.keras.preprocessing.sequence.pad_sequences(
#             summary_sequences[start:end], maxlen=SUMMARY_MAX_LENGTH, padding='post', truncating='post'
#         )

#         # generate a summary for each input sequence
#         predictions = model.predict(padded_text_sequences, batch_size=batch_size)
#         for j in range(len(predictions)):
#             # decode the predicted summary
#             summary = decode_summary_sequence(summary_tokenizer, predictions[j])
#             reference = decode_summary_sequence(summary_tokenizer, padded_summary_sequences[j])
#             rouge_score = rouge.get_scores(summary, reference, avg=True)['rouge-1']['f']
#             bleu_score = sentence_bleu([reference], summary)
#             meteor_score = meteor_score_fn(summary, reference)

#             rouge_scores.append(rouge_score)
#             bleu_scores.append(bleu_score)
#             meteor_scores.append(meteor_score)

#     avg_rouge_score = np.mean(rouge_scores)
#     avg_bleu_score = np.mean(bleu_scores)
#     avg_meteor_score = np.mean(meteor_scores)

#     return avg_rouge_score, avg_bleu_score, avg_meteor_score

In [ ]:
# def evaluate_model(model, text_tokenizer, text_sequences, summary_tokenizer, summary_sequences, batch_size):
#     rouge_scores = []
#     bleu_scores = []
#     meteor_scores = []
#     for i in range(0, len(text_sequences), batch_size):
#         text_sequence = text_sequences[i:i+batch_size]
#         summary_sequence = summary_sequences[i:i+batch_size]
#         encoder_output, forward_h, forward_c, backward_h, backward_c = model.layers[3].output
#         state_h = Concatenate()([forward_h, backward_h])
#         state_c = Concatenate()([forward_c, backward_c])
#         encoder_states = [state_h, state_c]
#         decoder_states = encoder_states
#         target_seq = np.zeros((len(text_sequence), 1))
#         target_seq[:, 0] = summary_tokenizer.word_index['start']
#         stop_condition = False
#         summary = ''
#         while not stop_condition:
#             output_tokens, h, c = model.layers[4].predict([tf.convert_to_tensor(target_seq)] + decoder_states)
#             sampled_token_index = np.argmax(output_tokens, axis=-1)
#             sampled_token = summary_tokenizer.index_word[int(sampled_token_index)]
#             if sampled_token != 'end':
#                 summary += ' ' + sampled_token
#             target_seq = np.zeros((len(text_sequence), 1))
#             target_seq[:, 0] = sampled_token_index
#             decoder_states = [h, c]
#             if sampled_token == 'end' or len(summary.split()) >= SUMMARY_MAX_LENGTH-1:
#                 stop_condition = True
#         text = ' '.join([text_tokenizer.index_word[int(i)] for i in text_sequence[0] if i > 0])
#         reference = ' '.join([summary_tokenizer.index_word[int(i)] for i in summary_sequence[0] if i > 0])
#         rouge_score = rouge.get_scores(summary, reference)[0]['rouge-l']['f']
#         bleu_score = sentence_bleu([reference.split()], summary.split(), weights=(0.25, 0.25, 0.25, 0.25))
#         meteor_score = single_meteor_score(reference, summary)
#         rouge_scores.append(rouge_score)
#         bleu_scores.append(bleu_score)
#         meteor_scores.append(meteor_score)
#     avg_rouge_score = sum(rouge_scores) / len(rouge_scores)
#     avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
#     avg_meteor_score = sum(meteor_scores) / len(meteor_scores)
#     return avg_rouge_score, avg_bleu_score, avg_meteor_score


In [ ]:
def predict_summary(model, text_tokenizer, text, summary_tokenizer, max_length, summary_max_length):
    # Tokenize the input text
    text_sequence = text_tokenizer.texts_to_sequences([text])
    text_sequence = pad_sequences(text_sequence, max_length)

    # Generate the summary
    predicted_summary = model.predict([text_sequence])[0]

    # Convert the predicted summary from integers to words
    predicted_summary = [summary_tokenizer.index_word[index] for index in predicted_summary if index > 0]
    predicted_summary = ' '.join(predicted_summary)
    return predicted_summary

In [ ]:
def test_model(model, text_tokenizer, summary_tokenizer, x_test, y_test):
    # Create lists to store predicted and actual summaries
    predicted_summaries = []
    actual_summaries = []

    # Loop through each test sample
    for i in range(len(x_test)):
        # Decode the predicted summary
        predicted_summary = decode_sequence(model, text_tokenizer, x_test[i])

        # Decode the actual summary
        actual_summary = decode_summary_sequence(summary_tokenizer, y_test[i])

        # Add the predicted and actual summaries to the lists
        predicted_summaries.append(predicted_summary)
        actual_summaries.append(actual_summary)

    # Compute evaluation metrics
    rouge_scores = [compute_rouge(actual_summaries[i], predicted_summaries[i]) for i in range(len(actual_summaries))]
    bleu_scores = [compute_bleu(actual_summaries[i], predicted_summaries[i]) for i in range(len(actual_summaries))]
    meteor_scores = [compute_meteor(actual_summaries[i], predicted_summaries[i]) for i in range(len(actual_summaries))]

    # Print the evaluation metrics
    print(f'ROUGE-1: {np.mean(rouge_scores):.4f}')
    print(f'BLEU: {np.mean(bleu_scores):.4f}')
    print(f'METEOR: {np.mean(meteor_scores):.4f}')


In [ ]:
def main(file_path):
    texts, summaries = load_data(file_path)
    text_tokenizer, text_sequences, summary_tokenizer, summary_sequences = tokenize(texts, summaries)
    padded_text_sequences, padded_summary_sequences = pad_sequences(text_sequences, summary_sequences, MAX_LENGTH, SUMMARY_MAX_LENGTH)
    train_text_sequences, train_summary_sequences, validation_text_sequences, validation_summary_sequences = split_data(padded_text_sequences, padded_summary_sequences)
    model = build_model(text_tokenizer.num_words, summary_tokenizer.num_words, EMBEDDING_DIM, MAX_LENGTH, SUMMARY_MAX_LENGTH)
    train_model(model, train_text_sequences, train_summary_sequences, validation_text_sequences, validation_summary_sequences, BATCH_SIZE, EPOCHS)
    avg_rouge_score, avg_bleu_score, avg_meteor_score = evaluate_model(model, text_tokenizer, text_sequences, summary_tokenizer, summary_sequences, BATCH_SIZE)
    print('Rouge score:', avg_rouge_score)
    print('Bleu score:', avg_bleu_score)
    print('Meteor score:', avg_meteor_score)

In [ ]:
main(r'/content/news_summary.csv')

29/29 [==============================] - 61s 2s/step - loss: 8.8437 - val_loss: 7.3717
Epoch 1/50
--------------------
Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_60 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 embedding_55 (Embedding)       (None, 50, 300)      3000000     ['input_60[0][0]']               
                                                                                                  
 input_61 (InputLayer)          [(None, 10)]         0           []                               
                                                                                                  
 bidirectional_39 (Bidirectiona  [(None, 50, 256),   439296      ['embe

<ipython-input-259-6929c4d6facf>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  encoder_input = np.array(text_sequence).reshape(1, len(text_sequence))


AttributeError: ignored

In [ ]:
# def fmain(file_path):
#     texts, summaries = load_data(file_path)
#     text_tokenizer, text_sequences, summary_tokenizer, summary_sequences = tokenize(texts, summaries)
#     padded_text_sequences, padded_summary_sequences = pad_sequences(text_sequences, summary_sequences, MAX_LENGTH, SUMMARY_MAX_LENGTH)
#     train_text_sequences, train_summary_sequences, validation_text_sequences, validation_summary_sequences = split_data(padded_text_sequences, padded_summary_sequences)
#     model = build_model(VOCAB_SIZE)
#     #text_tokenizer.num_words, summary_tokenizer.num_words, EMBEDDING_DIM, MAX_LENGTH, SUMMARY_MAX_LENGTH
#     # train_model(model, train_text_sequences, train_summary_sequences, validation_text_sequences, validation_summary_sequences, BATCH_SIZE, EPOCHS)
#     avg_rouge_score = evaluate_model(model, text_tokenizer, text_sequences, summary_tokenizer, summary_sequences, BATCH_SIZE)
#     ro_sc = compute_rouge()
#     print('Rouge score:', avg_rouge_score)
#     # print('Bleu score:', avg_bleu_score)
#     # print('Meteor score:', avg_meteor_score)
#     # predict_summary(model, text_tokenizer, text, summary_tokenizer, max_length, summary_max_length)

In [ ]:
def fmain(file_path):
    # Load the data
    text, summary = load_data(file_path)

    # Tokenize the text and summary
    text_tokenizer, summary_tokenizer = tokenize(text, summary)

    word_index = text_tokenizer.word_index
    words = [word for word in word_index]

    sum_index = summary_tokenizer.word_index
    sum = [word for word in sum_index]

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(text)
    # Pad the sequences
    padded_text_sequences, padded_summary_sequences = pad_sequences(words, sum, MAX_LENGTH, SUMMARY_MAX_LENGTH, tokenizer)

    # Split the data into training and validation sets
    train_text_sequences, train_summary_sequences, validation_text_sequences, validation_summary_sequences = split_data(padded_text_sequences, padded_summary_sequences)

    # Build the model
    model = build_model(len(text_tokenizer.word_index)+1)

    # Train the model
    # train_model(model, train_text_sequences, train_summary_sequences, validation_text_sequences, validation_summary_sequences)
    train_model(model, train_text_sequences, train_summary_sequences, validation_text_sequences, validation_summary_sequences, BATCH_SIZE, EPOCHS)

    # Test the model
    test_model(model, text_tokenizer, summary_tokenizer, validation_text_sequences, validation_summary_sequences)

In [ ]:
fmain('/content/news_summary.csv')

ValueError: ignored